In [1]:
IMG_PATH = '/data/amazon/train-jpg/'
IMG_EXT = '.jpg'
TRAIN_DATA = '/data/amazon/train_v2.csv'
SIZE = (64, 64)

In [2]:
import os
import gc # garbage collection
import bcolz
import cv2 
import numpy as np; import pandas as pd; import seaborn as sns
import matplotlib.pyplot as plt; import matplotlib.image as mpimg
import tensorflow as tf
from keras.models import Sequential
from keras.layers import *
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, History
# import vgg16
# import data_helper
# from data_helper import AmazonPreprocessor
# from kaggle_data.downloader import KaggleDataDownloader
from tqdm import tqdm

Using TensorFlow backend.


In [45]:
x = []; y = []
df_train = pd.read_csv(TRAIN_DATA)

In [46]:
flatten = lambda l: [item for sublist in l for item in sublist]
labels = list(set(flatten([l.split(' ') for l in df_train['tags'].values])))

label_map = {l: i for i, l in enumerate(labels)}
inv_label_map = {i: l for l, i in label_map.items()}

for f, tags in tqdm(df_train.values, miniters=1000):
    img = cv2.imread(f'{IMG_PATH}{f}{IMG_EXT}')
    targets = np.zeros(17)
    for t in tags.split(' '):
        targets[label_map[t]] = 1 
    x.append(cv2.resize(img, SIZE))
    y.append(targets)
    
y = np.array(y, np.uint8)
x = np.array(x, np.float16) / 255.

print(x.shape)
print(y.shape)

100%|██████████| 40479/40479 [00:53<00:00, 754.87it/s]


(40479, 64, 64, 3)
(40479, 17)


In [47]:
x.shape

(40479, 64, 64, 3)

In [49]:
split = 20000
x_train, x_valid = x[:split], x[split:] 
y_train, y_valid = y[:split], y[split:]

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(64, 64, 3)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(17, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(x_train, y_train,
          batch_size=128, epochs=50, verbose=1,
          validation_data=(x_valid, y_valid))

Train on 20000 samples, validate on 20479 samples
Epoch 1/50
20000/20000 [==============================] - 11s - loss: 0.2705 - acc: 0.8974 - val_loss: 0.2151 - val_acc: 0.9149
Epoch 2/50
20000/20000 [==============================] - 11s - loss: 0.2170 - acc: 0.9155 - val_loss: 0.1921 - val_acc: 0.9228
Epoch 3/50
20000/20000 [==============================] - 10s - loss: 0.2010 - acc: 0.9215 - val_loss: 0.1819 - val_acc: 0.9279
Epoch 4/50
20000/20000 [==============================] - 10s - loss: 0.1931 - acc: 0.9245 - val_loss: 0.1779 - val_acc: 0.9284
Epoch 5/50
20000/20000 [==============================] - 11s - loss: 0.1848 - acc: 0.9273 - val_loss: 0.1698 - val_acc: 0.9332
Epoch 6/50
20000/20000 [==============================] - 10s - loss: 0.1785 - acc: 0.9306 - val_loss: 0.1717 - val_acc: 0.9300
Epoch 7/50
20000/20000 [==============================] - 10s - loss: 0.1750 - acc: 0.9315 - val_loss: 0.1665 - val_acc: 0.9344
Epoch 8/50
20000/20000 [==============================

In [52]:
from sklearn.metrics import fbeta_score
p_valid = model.predict(x_valid, batch_size=128)
# print(y_valid)
# print(p_valid)
print(fbeta_score(y_valid, np.array(p_valid) > 0.2, beta=2, average='samples'))

0.854775001503
